In [4]:
import pandas as pd
from archetypal import settings
from schedules import mutate_timeseries
from shoeboxer.builder import ShoeBox, template_dict
from shoeboxer.shoebox_config import ShoeboxConfiguration
from utils.constants import SHADING_DIV_SIZE
import math
from pathlib import Path
import numpy as np

In [5]:
settings.energyplus_location = Path("C:\EnergyPlusV22-2-0")
settings.ep_version = "22.2.0"
df = pd.read_hdf(
    "data/hdf5/single-climate-zone/v1/nyc/monthly.hdf",
    key="batch_results",
)
features = df.index.to_frame(index=False)
features = features[features.error]
features = features.set_index("id")

for ix, row in features.iterrows():
    print("\n---", ix, "---")
    shoebox_config = ShoeboxConfiguration()
    shoebox_config.width = round(row.width, 4)
    shoebox_config.height = round(row.height, 4)
    shoebox_config.floor_2_facade = round(row.floor_2_facade, 4)
    shoebox_config.core_2_perim = round(row.core_2_perim, 4)
    shoebox_config.roof_2_footprint = round(row.roof_2_footprint, 4)
    shoebox_config.ground_2_footprint = round(row.ground_2_footprint, 4)
    shoebox_config.wwr = row.wwr
    shoebox_config.orientation = row.orientation
    shoebox_config.shading_vect = np.random.random(SHADING_DIV_SIZE) * math.pi / 3

    # MAKE FAKE SCHEDULES
    schedules = np.zeros((3, 21))
    # UNIRAND
    # Equipment
    schedules[0, 18] = 1  # hours per sample
    schedules[0, 17] = 24 * 7  # samples per pattern
    schedules[0, 16] = 0  # continuous
    # Occupancy
    schedules[1, 18] = 1  # hours per sample
    schedules[1, 17] = 24  # samples per pattern
    schedules[1, 16] = 0  # continuous
    # Lights
    schedules[2, 18] = 1  # hours per sample
    schedules[2, 17] = 22  # samples per pattern
    schedules[2, 16] = 0  # continuous
    # build timeseries
    scheds = mutate_timeseries(np.ones((3, 8760)), schedules, 0)

    epw = "data/epws/city_epws_indexed/cityidx_0001_USA_NY-New York Central Prk Obs Belv.725033_TMY3.epw"
    # epw = "data/epws/city_epws_indexed/cityidx_0119_USA_NY-New York City La Guard.725030_TMY--14732.epw"
    out_dir = Path("./shoeboxer/cache")
    d = template_dict(scheds)
    d["ventilation_mode"] = 1
    try:
        sb = ShoeBox(
            name="test",
            shoebox_config=shoebox_config,
            epw=epw,
            output_directory=out_dir,
            template_dict=d,
            change_summary=True,
        )
    except ValueError as e:
        if "is too small" in e.args[0]:
            continue
        else:
            raise e
    idf = sb.idf(run_simulation=True)
    errors, warnings = sb.error_report(idf)
    warnings = [
        w
        for w in warnings
        if "Output:Meter: invalid Key Name" not in w
        and "CheckUsedConstructions" not in w
        and "GetPurchasedAir" not in w
        and "The following Report Variables" not in w
        and "psysatfntemp" not in w.lower()
        and "GPU" not in w.lower()
    ]
    print(errors)
    print(warnings)


DEBUG:ShoeBox:Changed orientation of relative north to 0
DEBUG:ShoeBox:New R-val = 6.011662745098368 compared to desired 6
DEBUG:ShoeBox:Recalculating r-values...
DEBUG:ShoeBox:New R-val = 6.007006360593347 compared to desired 6
DEBUG:ShoeBox:New R-val = 2.0028807588075876 compared to desired 2
DEBUG:ShoeBox:Recalculating r-values...
DEBUG:ShoeBox:New R-val = 2.0120749426725033 compared to desired 2
DEBUG:ShoeBox:New R-val = 3.993656862745098 compared to desired 4
DEBUG:ShoeBox:Mechanical ventilation schedule: AllOn
INFO:shoeboxer.geometry_utils:New perimeter depth: 3.3175999999999997
INFO:shoeboxer.geometry_utils:New core depth: 3.616184
INFO:shoeboxer.geometry_utils:Shifting core origin to perimeter edge; moving -1.6824000000000003 m in y axis
INFO:shoeboxer.geometry_utils:Updating adiabatic surfaces.
INFO:shoeboxer.geometry_utils:Maximum shading point is 7.968160437656959
INFO:shoeboxer.geometry_utils:Placing shading around center point [2.165 0.   ]



--- d545a072-5dbb-46e5-ac66-7ed382f49422 ---


INFO:ShoeBox:Saved report as shoeboxer\cache\test_changereport.json
INFO:ShoeBox:Building idf for shoeboxer\cache\test.epjson
DEBUG:ShoeBox:Converting shoeboxer\cache\test.epjson to idf
INFO:ShoeBox:EnergyPlus Starting
INFO:ShoeBox:EnergyPlus, Version 22.2.0-c249759bad, YMD=2023.10.20 10:11
INFO:ShoeBox:Converted input file format. Exiting.
INFO:ShoeBox:EnergyPlus Run Time=00hr 00min  0.07sec
INFO:ShoeBox:Command executed successfully.
C:\EnergyPlusV22-2-0\ExpandObjects.EXE #0-test.idf: 3.00it [00:00, 11.1it/s]
C:\EnergyPlusV22-2-0\energyplus.exe test.idf: 55it [00:03, 14.51it/s]
INFO:ShoeBox:HEATING/COOLING EUI
INFO:ShoeBox:IndexGroup  KeyValue                   Name                                            
System      CORE IDEAL LOADS AIR       Zone Ideal Loads Supply Air Total Heating Energy     7.145528
                                       Zone Ideal Loads Supply Air Total Cooling Energy    10.809550
            PERIMETER IDEAL LOADS AIR  Zone Ideal Loads Supply Air Total Heat

[]
['** Warning ** Temperature out of range [-100. to 200.] (PsyPsatFnTemp)']

--- 7b269ed4-e71b-42c7-8166-1b6e4769970d ---

--- 6eb9e58e-826f-4cc5-96bc-533c66734a63 ---


INFO:shoeboxer.geometry_utils:New perimeter depth: 2.0178
INFO:shoeboxer.geometry_utils:New core depth: 0.12106799999999998
ERROR:shoeboxer.geometry_utils:Depth is too small. Dropping shoebox.
DEBUG:ShoeBox:Changed orientation of relative north to 270
DEBUG:ShoeBox:New R-val = 6.011662745098368 compared to desired 6
DEBUG:ShoeBox:Recalculating r-values...
DEBUG:ShoeBox:New R-val = 6.007006360593347 compared to desired 6
DEBUG:ShoeBox:New R-val = 2.0028807588075876 compared to desired 2
DEBUG:ShoeBox:Recalculating r-values...
DEBUG:ShoeBox:New R-val = 2.0120749426725033 compared to desired 2
DEBUG:ShoeBox:New R-val = 3.993656862745098 compared to desired 4
DEBUG:ShoeBox:Mechanical ventilation schedule: AllOn
INFO:shoeboxer.geometry_utils:New perimeter depth: 9.6579
INFO:shoeboxer.geometry_utils:New core depth: 7.629741
INFO:shoeboxer.geometry_utils:Shifting core origin to perimeter edge; moving 4.6579 m in y axis
INFO:shoeboxer.geometry_utils:Updating adiabatic surfaces.
INFO:shoeboxer.

...

INFO:shoeboxer.geometry_utils:New perimeter depth: 2.366
INFO:shoeboxer.geometry_utils:New core depth: 0.16562000000000002
ERROR:shoeboxer.geometry_utils:Depth is too small. Dropping shoebox.
DEBUG:ShoeBox:Changed orientation of relative north to 180
DEBUG:ShoeBox:New R-val = 6.011662745098368 compared to desired 6
DEBUG:ShoeBox:Recalculating r-values...
DEBUG:ShoeBox:New R-val = 6.007006360593347 compared to desired 6
DEBUG:ShoeBox:New R-val = 2.0028807588075876 compared to desired 2
DEBUG:ShoeBox:Recalculating r-values...
DEBUG:ShoeBox:New R-val = 2.0120749426725033 compared to desired 2
DEBUG:ShoeBox:New R-val = 3.993656862745098 compared to desired 4
DEBUG:ShoeBox:Mechanical ventilation schedule: AllOn
INFO:shoeboxer.geometry_utils:New perimeter depth: 2.8985
INFO:shoeboxer.geometry_utils:New core depth: 0.144925
ERROR:shoeboxer.geometry_utils:Depth is too small. Dropping shoebox.
DEBUG:ShoeBox:Changed orientation of relative north to 90
DEBUG:ShoeBox:New R-val = 6.011662745098368 


--- be9d2ed7-aeff-4dfe-b630-120ef87d937b ---

--- 13e99f8b-3bb6-4c05-b90a-1c2992d1c4c9 ---


INFO:shoeboxer.geometry_utils:New core depth: 0.22585199999999997
ERROR:shoeboxer.geometry_utils:Depth is too small. Dropping shoebox.
DEBUG:ShoeBox:Changed orientation of relative north to 90
DEBUG:ShoeBox:New R-val = 6.011662745098368 compared to desired 6
DEBUG:ShoeBox:Recalculating r-values...
DEBUG:ShoeBox:New R-val = 6.007006360593347 compared to desired 6
DEBUG:ShoeBox:New R-val = 2.0028807588075876 compared to desired 2
DEBUG:ShoeBox:Recalculating r-values...
DEBUG:ShoeBox:New R-val = 2.0120749426725033 compared to desired 2
DEBUG:ShoeBox:New R-val = 3.993656862745098 compared to desired 4
DEBUG:ShoeBox:Mechanical ventilation schedule: AllOn



--- 10209c9b-80c5-49af-93c6-00a22917132c ---


KeyboardInterrupt: 